In [ ]:
# BLOCK: HYBRID RECOMMENDATION ENGINE (FINAL PRODUCT - FIXED)
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# ====================================================
# 1. KHỞI TẠO VÀ LOAD TÀI NGUYÊN
# ====================================================
print("⚙️ Đang khởi động hệ thống Hybrid...")

# A. Load Metadata & Vector (Content-Based)
try:
    df_vectors = pd.read_csv('music_vector_database.csv')
    # Lấy các cột vector (emb_0, emb_1...)
    vector_cols = [c for c in df_vectors.columns if c.startswith('emb_')]
    matrix_vectors = df_vectors[vector_cols].values
    print("✅ Đã load Content-Based Database.")
except Exception as e:
    print(f"❌ Lỗi load CSV: {e}")

# B. Load Model & Encoders (Collaborative Filtering)
try:
    # --- FIX 1: Thêm compile=False ---
    ncf_model = tf.keras.models.load_model('ncf_model_sampled.h5', compile=False)
    
    with open('user_encoder.pkl', 'rb') as f:
        user_encoder = pickle.load(f)
    with open('track_encoder.pkl', 'rb') as f:
        track_encoder = pickle.load(f)
    print("✅ Đã load Collaborative Filtering Model.")
except Exception as e:
    print(f"⚠️ Lỗi load model NCF: {e}")
    ncf_model = None

# ====================================================
# 2. ĐỊNH NGHĨA CÁC HÀM GỢI Ý
# ====================================================

def recommend_for_guest(song_name, top_k=10):
    """Chiến thuật: Content-Based (Dùng khi không biết User là ai)"""
    print(f"\n🔍 [GUEST MODE] Đang tìm bài hát giống: '{song_name}'...")
    
    # Tìm bài hát trong database
    song_row = df_vectors[df_vectors['name'].str.contains(song_name, case=False, na=False)]
    if song_row.empty:
        return "❌ Không tìm thấy bài hát này trong hệ thống."
    
    # Lấy vector bài hát mẫu
    target_vec = song_row.iloc[0][vector_cols].values.reshape(1, -1)
    
    # Tính độ tương đồng
    sim_scores = cosine_similarity(target_vec, matrix_vectors).flatten()
    
    # Lấy Top K
    top_indices = sim_scores.argsort()[::-1][1:top_k+1]
    
    # --- FIX 2: Bỏ 'year' vì file dữ liệu không có cột này ---
    cols_to_show = ['name', 'artist']
    if 'year' in df_vectors.columns: cols_to_show.append('year')
    
    results = df_vectors.iloc[top_indices][cols_to_show].copy()
    results['score'] = sim_scores[top_indices]
    return results

def recommend_for_user(user_id_raw, top_k=10):
    """Chiến thuật: Collaborative Filtering (Dùng cho User đã có lịch sử)"""
    print(f"\n👤 [USER MODE] Đang phân tích gu của User ID: {user_id_raw[:10]}...")
    
    if ncf_model is None:
        return "⚠️ Model NCF chưa được load thành công."

    # 1. Kiểm tra User có tồn tại không
    try:
        user_idx = user_encoder.transform([user_id_raw])[0]
    except ValueError:
        print("   -> ⚠️ User này MỚI TINH (chưa có trong tập train).")
        return None # Trả về None để chuyển sang chế độ Guest

    # 2. Dự đoán
    all_track_idxs = np.arange(len(track_encoder.classes_))
    user_idxs = np.full(len(all_track_idxs), user_idx)
    
    predictions = ncf_model.predict([user_idxs, all_track_idxs], batch_size=4096, verbose=0).flatten()
    
    # 3. Lấy Top K
    top_indices = predictions.argsort()[::-1][:top_k]
    top_track_idxs = all_track_idxs[top_indices]
    top_track_ids = track_encoder.inverse_transform(top_track_idxs)
    
    # 4. Trả về kết quả
    results = df_vectors[df_vectors['track_id'].isin(top_track_ids)].copy()
    
    # --- FIX 2: Bỏ 'year' ở đây luôn ---
    cols_to_show = ['name', 'artist']
    if 'year' in df_vectors.columns: cols_to_show.append('year')

    return results[cols_to_show]

# ====================================================
# 3. HÀM TỔNG HỢP (HYBRID SYSTEM)
# ====================================================
def hybrid_engine(user_id=None, song_input=None):
    # CASE 1: Nếu có User ID -> Thử dùng NCF
    if user_id:
        rec_list = recommend_for_user(user_id)
        if rec_list is not None:
            print("✅ Đã tìm thấy User trong hệ thống! Dùng Collaborative Filtering.")
            return rec_list
        else:
            print("🔄 Chuyển sang chế độ Content-Based (Cold Start)...")
    
    # CASE 2: Nếu không có User ID hoặc User mới -> Dùng Content-Based
    if song_input:
        print(f"✅ Gợi ý dựa trên bài hát bạn nhập: {song_input}")
        return recommend_for_guest(song_input)
    
    return "❌ Vui lòng nhập User ID hoặc tên bài hát để bắt đầu."

# --- Hàm mới: Gợi ý bài hát liên quan dựa trên hành vi người dùng (Item-to-Item CF) ---


# ====================================================
# 4. CHẠY THỬ NGHIỆM
# ====================================================

# --- TEST 1: Lấy ngẫu nhiên 1 User từ file lịch sử để test ---
try:
    df_history_test = pd.read_csv('User Listening History.csv', nrows=100)
    random_user = df_history_test['user_id'].iloc[5] 
    
    print(f"\n🧪 TEST 1: Gợi ý cho User cũ ({random_user})")
    results_user = hybrid_engine(user_id=random_user)
    print(results_user)
except:
    print("⚠️ Không tìm thấy file lịch sử để test user.")

# --- TEST 2: Giả lập User mới ---
print("\n" + "="*50)
print("🧪 TEST 2: Gợi ý cho User mới (Thích bài 'Yesterday')")
results_guest = hybrid_engine(user_id="User_Moi_Tinh", song_input="Yesterday")
print(results_guest)

⚙️ Đang khởi động hệ thống Hybrid...
✅ Đã load Content-Based Database.
✅ Đã load Collaborative Filtering Model.

🧪 TEST 1: Gợi ý cho User cũ (b80344d063b5ccb3212f76538f3d9e43d87dca9e)

👤 [USER MODE] Đang phân tích gu của User ID: b80344d063...
✅ Đã tìm thấy User trong hệ thống! Dùng Collaborative Filtering.
                        name                  artist
941    My Boy Builds Coffins  Florence + the Machine
1125                     Dig                 Incubus
1709         A Comet Appears               The Shins
2499                Promises                    Nero
2673               Pap Smear         Crystal Castles
4866                The Food                  Common
5526                New Soul               Yael Naim
5819               Sun Giant             Fleet Foxes
25621                Ray Gun    The Bird and the Bee
38473                    Soy             Gipsy Kings

🧪 TEST 2: Gợi ý cho User mới (Thích bài 'Yesterday')

👤 [USER MODE] Đang phân tích gu của User ID: User_Moi